# **LoRA_Inferance_and_GridSearch**

**Setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Loading Stable Diffusion with LoRA Weights for SVG-Optimized Image Generation**

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
    # device="cuda"
).to("cuda")

# replace the path with the path containing the LoRA weights
pipe.load_lora_weights("/content/drive/MyDrive/Loras/StableDiffusionForSVGableImagesGeneration/output/StableDiffusionForSVGableImagesGeneration-03.safetensors")


**Generating and Saving an Image with Stable Diffusion**

In [ ]:
pipe.disable_lora()
prompt = "A black siluhette of a man with white background"
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": 1.5}, generator=torch.manual_seed(1)
).images[0]
image

# Save the image
image_path = "/content/drive/MyDrive/images/silhouette_man_with_hat.png"  # Specify the file name
image.save(image_path)

print(f"Image saved to {image_path}")

**Generating and Presenting an Image with Stable Diffusion**

In [ ]:
pipe.enable_lora()
prompt = "A simple black painting of a cat, solid white backd white siground"
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": 1.2}, generator=torch.manual_seed(1)
).images[0]
image

**Remove Background of an Image**

In [ ]:
import cv2
import numpy as np

image_path = "/content/drive/MyDrive/images/image_tree.png"
grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

gray_threshold_low = 20
gray_threshold_high = 200
background_mask = cv2.inRange(grayscale_image, gray_threshold_low, gray_threshold_high)
grayscale_image[background_mask > 0] = 255

_, binary_image = cv2.threshold(grayscale_image, 128, 255, cv2.THRESH_BINARY)

num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

min_size = 50  # Minimum size of components to keep
filtered_image = np.zeros_like(binary_image)

for i in range(1, num_labels):
    if stats[i, cv2.CC_STAT_AREA] >= min_size:
        filtered_image[labels == i] = 255

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

closed_image = cv2.morphologyEx(filtered_image, cv2.MORPH_CLOSE, kernel)  # Fill small holes
smoothed_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel)  # Remove noise

cv2.imwrite("/content/drive/MyDrive/images/smoothed_image_tree.png", smoothed_image)
# cv2.imshow("Smoothed Image", smoothed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# **Full Pipeline**

**Installations**

In [ ]:
!apt-get install potrace
!pip install potracer[cli]
!pip install tiktoken

**Utils Functions**

Imports

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import subprocess

Remove Background Function

In [ ]:
def remove_background(image_path):
    """
    Removes the background of an image, isolates significant components, and smooths the result.

    Args:
        image_path (str): Path to the input grayscale image.

    Returns:
        np.ndarray: Processed image with background removed and components smoothed.
    """
    grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if grayscale_image is None:
        raise ValueError(f"Image not found at path: {image_path}")

    gray_threshold_low = 20
    gray_threshold_high = 200
    background_mask = cv2.inRange(grayscale_image, gray_threshold_low, gray_threshold_high)
    grayscale_image[background_mask > 0] = 255

    _, binary_image = cv2.threshold(grayscale_image, 128, 255, cv2.THRESH_BINARY)

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

    min_size = 50
    filtered_image = np.zeros_like(binary_image)

    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= min_size:
            filtered_image[labels == i] = 255

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

    closed_image = cv2.morphologyEx(filtered_image, cv2.MORPH_CLOSE, kernel)
    smoothed_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel)

    return smoothed_image


Convert Image to SVG Function

In [ ]:
def convert_to_svg(image_path, svg_output_dir):
    """
    Converts an image to SVG format using the following steps:
    1. Resizes the image to 500x500 pixels.
    2. Converts the resized image to grayscale.
    3. Saves the grayscale image as a BMP file.
    4. Converts the BMP file to SVG using Potrace.

    Parameters:
        image_path (str): Path to the input image file.
        svg_output_dir (str): Path to the directory where the SVG file will be saved.

    Returns:
        str: Path to the saved SVG file.

    Raises:
        Exception: If any step in the process fails.
    """
    try:
        os.makedirs(svg_output_dir, exist_ok=True)

        img = Image.open(image_path)
        base_name = os.path.splitext(os.path.basename(image_path))[0]

        img_resized = img.resize((500, 500), Image.LANCZOS)

        img_gray = img_resized.convert('L')

        bmp_path = os.path.join(svg_output_dir, f'{base_name}.bmp')
        img_gray.save(bmp_path)

        svg_path = os.path.join(svg_output_dir, f'{base_name}.svg')
        subprocess.run(['potrace', bmp_path, '-s', '-o', svg_path], check=True)

        print(f'SVG file saved to {svg_path}')
        return svg_path

    except Exception as e:
        raise Exception(f"An error occurred while processing {image_path}: {e}")


**Grid Search Pipeline:**

Imports and Defnitions

In [ ]:
import os
import pandas as pd
import torch
from itertools import product
from PIL import Image
import cv2
import numpy as np
import subprocess
import tiktoken

# Hyperparameters and prompts
prompts = [
    "A minimalistic black silhouette of a {object} on a plain white background, with clean lines and no additional details.",
    "A simple black outline of a {object}, completely filled in, positioned on a white background for a sharp contrast.",
    "A sleek black silhouette of a {object}, centered on a white background, designed to emphasize the shape of the object.",
    "A bold, black silhouette representation of a {object}, isolated on a plain white background for clarity.",
    "A clean, vector-style black silhouette of a {object} against a pure white background, with no extraneous details.",
    "A minimalistic black shadow image of a {object}, rendered against a bright white background.",
    "A sharp-edged black silhouette of a {object}, perfectly centered and framed by a simple white backdrop.",
]

objects = ["tree", "cat", "bicycle", "airplane", "chair", "teapot", "guitar", "candle", "elephant", "lighthouse"]
num_inference_steps_range = [20, 30, 40]
scale_range = [1.0, 1.2, 1.5]
seeds = [1, 42, 99]

output_dir = "/content/drive/MyDrive/StableDiffusion-v1.5-outputs/train/images"
svg_output_dir = "/content/drive/MyDrive/StableDiffusion-v1.5-outputs/train/svgs"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(svg_output_dir, exist_ok=True)


Count Tokens Function

In [ ]:
def count_tokens(svg_content):
    encoding = tiktoken.encoding_for_model("gpt-4")
    return len(encoding.encode(svg_content))


Pipeline Execution

In [ ]:
results = []
for prompt_template, num_inference_steps, scale, seed in product(prompts, num_inference_steps_range, scale_range, seeds):
    generator = torch.manual_seed(seed)
    pos_count = 0
    for obj in objects:
      prompt = prompt_template.format(object=obj)
      try:
        image = pipe(prompt, num_inference_steps=num_inference_steps, cross_attention_kwargs={"scale": scale}, generator=generator).images[0]
        image_filename = f"{obj}_steps_{num_inference_steps}_scale_{scale}_seed_{seed}.png"
        image_path = os.path.join(output_dir, image_filename)
        image.save(image_path)

        smoothed_image = remove_background(image_path)
        processed_image_path = image_path.replace(".png", "_processed.png")
        cv2.imwrite(processed_image_path, smoothed_image)

        svg_path = convert_to_svg(processed_image_path, svg_output_dir)

        with open(svg_path, "r") as svg_file:
            svg_content = svg_file.read()
        token_count = count_tokens(svg_content)

        if token_count <= 8000:
            pos_count += 1

      except Exception as e:
        print(f"An error occurred for {obj} with seed {seed}: {e}")

    num_objects = len(objects)
    positive_rate = pos_count / num_objects

    results.append({
        "Prompt": prompt_template,
        "Inference Steps": num_inference_steps,
        "Scale": scale,
        "Seed": seed,
        "Positives Rate": positive_rate,
    })

    df = pd.DataFrame(results)

    # Save the results to an Excel file
    excel_file_path = "/content/drive/MyDrive/StableDiffusion-v1.5-outputs/train/results.xlsx"
    df.to_excel(excel_file_path, index=False)

    print(f"Results saved to {excel_file_path}")

**Images Generation Pipeline:**

Imports and Definitions

In [ ]:
import os
import torch
import cv2
import subprocess
from PIL import Image
import numpy as np


output_images_dir = "/content/drive/mydrive/LargeDataset/Images"
output_svgs_dir = "/content/drive/mydrive/LargeDataset/SVG"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_svgs_dir, exist_ok=True)

num_inference_steps = 30
scale = 1.5
seed = 1

prompts = [
    "A black silhouette of a man with a white background",
    "A simple black outline of a dog against a plain white background",
    "A minimalistic black figure of a tree, standing on a white canvas",
    "A black figure of a cat with a white background",
    "A simple silhouette of a car on a white background",
    "A minimalistic black bicycle silhouette on a white background"
]


Generate Image and Process Function

In [ ]:
def generate_image_and_process(prompt, num_inference_steps, scale, seed):
    """
    Generate image, process background, and save to respective directories.
    """
    generator = torch.manual_seed(seed)

    try:
        # Generate image
        image = pipe(prompt, num_inference_steps=num_inference_steps, cross_attention_kwargs={"scale": scale}, generator=generator).images[0]

        # Save generated image initially (before background removal)
        temp_image_path = "/tmp/temp_image.png"
        image.save(temp_image_path)

        # Remove background from image
        smoothed_image = remove_background(temp_image_path)

        # Save processed image after background removal
        processed_image_path = os.path.join(output_images_dir, f"{prompt.replace(' ', '_')}_steps_{num_inference_steps}_scale_{scale}_seed_{seed}_processed.png")
        cv2.imwrite(processed_image_path, smoothed_image)

        # Convert the processed image to SVG
        svg_path = convert_to_svg(processed_image_path, output_svgs_dir)

        print(f"Processed image saved as {processed_image_path} and SVG as {os.path.basename(svg_path)}")

    except Exception as e:
        print(f"Error processing prompt '{prompt}' with seed {seed}: {e}")


 Loop to generate and process images based on the list of prompts

In [ ]:
for prompt in prompts:
    generate_image_and_process(prompt, num_inference_steps, scale, seed)

print(f"Finished processing images.")